In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.5'
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,341 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,714 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("SELECT ROUND(AVG(price),2),YEAR(date) FROM home_sales WHERE bedrooms = 4 GROUP BY YEAR(date) ORDER BY YEAR(date)").show()


+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|            300263.7|      2019|
|           298353.78|      2020|
|           301819.44|      2021|
|           296363.88|      2022|
+--------------------+----------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("SELECT ROUND(AVG(price),2),date_built FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built ORDER BY date_built").show()

+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           292859.62|      2010|
|           291117.47|      2011|
|           293683.19|      2012|
|           295962.27|      2013|
|           290852.27|      2014|
|            288770.3|      2015|
|           290555.07|      2016|
|           292676.79|      2017|
+--------------------+----------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("SELECT ROUND(AVG(price),2),date_built FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors=2 AND sqft_living >= 2000 GROUP BY date_built ORDER BY date_built").show()

+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           285010.22|      2010|
|           276553.81|      2011|
|           307539.97|      2012|
|           303676.79|      2013|
|           298264.72|      2014|
|           297609.97|      2015|
|            293965.1|      2016|
|           280317.58|      2017|
+--------------------+----------+



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) FROM home_sales GROUP BY view HAVING ROUND(AVG(price),2) >= 350000 ORDER BY view DESC").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  99|          1061201.42|
|  98|          1053739.33|
|  97|          1129040.15|
|  96|          1017815.92|
|  95|           1054325.6|
|  94|           1033536.2|
|  93|          1026006.06|
|  92|           970402.55|
|  91|          1137372.73|
|  90|          1062654.16|
|  89|          1107839.15|
|  88|          1031719.35|
|  87|           1072285.2|
|  86|          1070444.25|
|  85|          1056336.74|
|  84|          1117233.13|
|  83|          1033965.93|
|  82|           1063498.0|
|  81|          1053472.79|
|  80|           991767.38|
+----+--------------------+
only showing top 20 rows

--- 0.7642908096313477 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")


DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) FROM home_sales GROUP BY view HAVING ROUND(AVG(price),2) >= 350000 ORDER BY view DESC").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  99|          1061201.42|
|  98|          1053739.33|
|  97|          1129040.15|
|  96|          1017815.92|
|  95|           1054325.6|
|  94|           1033536.2|
|  93|          1026006.06|
|  92|           970402.55|
|  91|          1137372.73|
|  90|          1062654.16|
|  89|          1107839.15|
|  88|          1031719.35|
|  87|           1072285.2|
|  86|          1070444.25|
|  85|          1056336.74|
|  84|          1117233.13|
|  83|          1033965.93|
|  82|           1063498.0|
|  81|          1053472.79|
|  80|           991767.38|
+----+--------------------+
only showing top 20 rows

--- 0.645094633102417 seconds ---


In [16]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').mode('overwrite').parquet('home_sales_partitioned')

In [17]:
# 11. Read the parquet formatted data.

parquet_df = spark.read.parquet('home_sales_partitioned')
parquet_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [18]:
# 12. Create a temporary table for the parquet data.

parquet_df.createOrReplaceTempView('parquet_temp_home')

In [20]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) FROM home_sales GROUP BY view HAVING ROUND(AVG(price),2) >= 350000 ORDER BY view DESC").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  99|          1061201.42|
|  98|          1053739.33|
|  97|          1129040.15|
|  96|          1017815.92|
|  95|           1054325.6|
|  94|           1033536.2|
|  93|          1026006.06|
|  92|           970402.55|
|  91|          1137372.73|
|  90|          1062654.16|
|  89|          1107839.15|
|  88|          1031719.35|
|  87|           1072285.2|
|  86|          1070444.25|
|  85|          1056336.74|
|  84|          1117233.13|
|  83|          1033965.93|
|  82|           1063498.0|
|  81|          1053472.79|
|  80|           991767.38|
+----+--------------------+
only showing top 20 rows

--- 0.5100107192993164 seconds ---


In [21]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")


DataFrame[]

In [22]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False